In [64]:
# %pip install tensorflow==2.16.1

In [2]:
#https://www.tensorflow.org/tutorials/load_data/images
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#resnet importalasa
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions


In [3]:
batch_size = 32
img_height = 128
img_width = 128

In [4]:
data_dir = "Train1"
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 53172 files belonging to 62 classes.
Using 42538 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 53172 files belonging to 62 classes.
Using 10634 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['Sample001', 'Sample002', 'Sample003', 'Sample004', 'Sample005', 'Sample006', 'Sample007', 'Sample008', 'Sample009', 'Sample010', 'Sample011', 'Sample012', 'Sample013', 'Sample014', 'Sample015', 'Sample016', 'Sample017', 'Sample018', 'Sample019', 'Sample020', 'Sample021', 'Sample022', 'Sample023', 'Sample024', 'Sample025', 'Sample026', 'Sample027', 'Sample028', 'Sample029', 'Sample030', 'Sample031', 'Sample032', 'Sample033', 'Sample034', 'Sample035', 'Sample036', 'Sample037', 'Sample038', 'Sample039', 'Sample040', 'Sample041', 'Sample042', 'Sample043', 'Sample044', 'Sample045', 'Sample046', 'Sample047', 'Sample048', 'Sample049', 'Sample050', 'Sample051', 'Sample052', 'Sample053', 'Sample054', 'Sample055', 'Sample056', 'Sample057', 'Sample058', 'Sample059', 'Sample060', 'Sample061', 'Sample062']


In [7]:
# transfer learning: egy tanított modell továbbtanítása
# https://medium.com/@bravinwasike18/building-a-deep-learning-model-with-keras-and-resnet-50-9dd6f4eb3351
imported_model= tf.keras.applications.ResNet50(include_top=False,
input_shape=(128,128,3),# azert egy az utso parameter mert egy csatorna van a kepben a fekete
pooling='avg',classes=62,
weights='imagenet')
for layer in imported_model.layers:
    layer.trainable=False

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input

# Előre betanított modell (pl. ResNet50)
base_model = imported_model

# Ne adjuk újra hozzá az Input réteget a base_model-hez
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(62, activation='softmax')(x)

# Modell definiálása
dnn_model = Model(inputs=base_model.input, outputs=output)

In [9]:
# dnn_model.summary()

In [10]:
# teljesítmény és pontosság növelése a tanitási fázisban adam modellel
from tensorflow.keras.optimizers import Adam
dnn_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
results = dnn_model.fit(
train_ds,
validation_data=val_ds,
epochs=10 # egy tanitasi folyamat
)

Epoch 1/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1238s 923ms/step - accuracy: 0.6353 - loss: 1.3967 - val_accuracy: 0.7894 - val_loss: 0.6295
Epoch 2/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1325s 996ms/step - accuracy: 0.8188 - loss: 0.5281 - val_accuracy: 0.8238 - val_loss: 0.5018
Epoch 3/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 7064s 5s/step - accuracy: 0.8497 - loss: 0.4074 - val_accuracy: 0.8514 - val_loss: 0.4254
Epoch 4/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1155s 868ms/step - accuracy: 0.8663 - loss: 0.3447 - val_accuracy: 0.8415 - val_loss: 0.4626
Epoch 5/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1169s 879ms/step - accuracy: 0.8741 - loss: 0.3144 - val_accuracy: 0.8524 - val_loss: 0.4228
Epoch 6/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1157s 870ms/step - accuracy: 0.8872 - loss: 0.2852 - val_accuracy: 0.8546 - val_loss: 0.4454
Epoch 7/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 1162s 874ms/step - accuracy: 0.8970 - loss: 0.2567 - val_accuracy: 0.8599 - val_loss: 0.4390
Epoch 8/10
1330/1330 ━━━━━━━━━━━━━━━━━━━━ 2263s 2s/step - 

In [12]:
# pip install opencv-python

In [13]:
# Tesztképek beolvasása
import cv2
import tqdm
import glob
test_images = [cv2.imread(file) for file in tqdm.tqdm(glob.glob("test/*.png"), desc = "Betoltes") ]
#resized_test_images = [ cv2.resize(image,(128,128)) for image in test_images] #atemretezes 128*128

'''image=np.expand_dims(image_resized,axis=0)
print(image.shape)'''

Betoltes: 100%|██████████| 7100/7100 [00:03<00:00, 1944.44it/s]


'image=np.expand_dims(image_resized,axis=0)\nprint(image.shape)'

In [14]:
test_images[1000].shape

(128, 128, 3)

In [15]:
'''for img in tqdm.tqdm(test_images, desc="teszteles"):
    # Resize the image to 128x128 with 3 channels
    image_resized = cv2.resize(img, (128, 128))

    #Add a batch dimension egyszerre hany kepet kezel
    #az alapvetoen 3d(magassag,szelesseg,melyseg) kephez meg egy dimenziot (hany kepbol dolgozzon,hanyat kezeljen egyszerre)
    image = np.expand_dims(image_resized, axis=0)  # Shape becomes (1, 128, 128, 3)
    #print(image.)
    model_pred=dnn_model.predict(image)#predikcio a modell alapjan
    predicted_class=class_names[np.argmax(model_pred)]#class nev kikeresese
    print("The predicted category is", predicted_class)'''

'for img in tqdm.tqdm(test_images, desc="teszteles"):\n    # Resize the image to 128x128 with 3 channels\n    image_resized = cv2.resize(img, (128, 128))\n\n    #Add a batch dimension egyszerre hany kepet kezel\n    #az alapvetoen 3d(magassag,szelesseg,melyseg) kephez meg egy dimenziot (hany kepbol dolgozzon,hanyat kezeljen egyszerre)\n    image = np.expand_dims(image_resized, axis=0)  # Shape becomes (1, 128, 128, 3)\n    #print(image.)\n    model_pred=dnn_model.predict(image)#predikcio a modell alapjan\n    predicted_class=class_names[np.argmax(model_pred)]#class nev kikeresese\n    print("The predicted category is", predicted_class)'

In [16]:
# txt-be írás
f = open("predictions.txt", "w")
f.write("class;TestImage")
sample=0

In [17]:
for img in tqdm.tqdm(test_images,desc="teszteles"):
    sample+=1
    # Resize the image to 128x128 with 3 channels
    image_resized = cv2.resize(img, (128, 128))

    #Add a batch dimension
    image = np.expand_dims(image_resized, axis=0)  # Shape becomes (1, 128, 128, 3)

    model_pred=dnn_model.predict(image)
    predicted_class=class_names[np.argmax(model_pred)]
    f.write(predicted_class[-2:]+";Test"+str(sample).zfill(4)+"\n")
    #print(predicted_class[-2:]+";Test"+str(sample).zfill(4))
f.close()

teszteles:   0%|          | 0/7100 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


teszteles:   0%|          | 1/7100 [00:02<5:45:18,  2.92s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


teszteles:   0%|          | 2/7100 [00:03<2:30:00,  1.27s/it]

...

teszteles: 100%|█████████▉| 7098/7100 [22:14<00:00,  6.12it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


teszteles: 100%|█████████▉| 7099/7100 [22:14<00:00,  5.96it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


teszteles: 100%|██████████| 7100/7100 [22:14<00:00,  5.32it/s]


In [18]:
'''import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")'''

'import matplotlib.pyplot as plt\n\nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n  for i in range(9):\n    ax = plt.subplot(3, 3, i + 1)\n    plt.imshow(images[i].numpy().astype("uint8"))\n    plt.title(class_names[labels[i]])\n    plt.axis("off")'

Using tf.data for finer control - eddig van meg>>
Holnap>> D:\Egyetemi programozás\5. félév\lua\GyakorlatokXhet\vitmav45\06\Keras_MNIST_CNN.ipynb
https://www.tensorflow.org/tutorials/images/classification
